In [ ]:

#  Here we demonstrate the following Dask DataFrame techniques;
#
#     .  Test, How to fetch data from a graph
#


#  Part 01:  Setup, generate data

In [ ]:

NUM_PARTITIONS  = 3
NUM_ROWS        = 40000

print("--")


In [ ]:

import numpy as np
import pandas as pd
   #
import dask.dataframe as dd
   #
from dask.dataframe import from_pandas

from tabulate import tabulate

print("--")


In [ ]:


#  Generates (NUM_ROWS) rows of data
#

l_row = {"col1": list(), "col2": list()}
   #
for l_cntr in range(NUM_ROWS):
   l_row["col1"].append(l_cntr)
   l_row["col2"].append("Item number: " + str(l_cntr))
    
pd_data = pd.DataFrame(l_row, columns = ["col1", "col2"])
dd_data = from_pandas(pd_data, npartitions = NUM_PARTITIONS)


l_cntr = 0
   #
for l_each in dd_data.itertuples():
   l_cntr += 1
      #
   if (l_cntr <5):
      print("Row (%d) of (%d) total rows.   Col1: (%4d)   Col2: (%s)" % (l_cntr, len(dd_data), l_each.col1, l_each.col2))
        
print("--")



#  Part 02:  Create a graph

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph" 

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPHS

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


#  Part 03:  Import into the graph

In [ ]:

with import_data.DataFrameImporter(my_graph) as df_importer:
    
   #  Movies
   #
   df_importer.nodes_dataframe(
      dd_data[["col1", "col2"]],
      id_column  = "col1",
      id_space   = "dd_data"
      )

   df_importer.insert()


print("--")


In [ ]:

display("Number of Graph Nodes: %d" % (my_graph.num_nodes()))
display("Number of Graph Edges: %s" % (my_graph.num_edges()))

#  Sample outpt,
#
#     'Number of Graph Nodes: 40000'
#     'Number of Graph Edges: 0'


#  Part 04:  Run a traversal, return results

In [ ]:

#  Aggregates, by definition, can return only one row. We use a slightly different
#  technique to get the return value.
#

l_query  = """
   MATCH (n) 
   RETURN COUNT(*) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]


print(l_count)
print(type(l_count))

#  Sample output,
#
#     40000
#     <class 'numpy.int64'>


In [ ]:

#  Returning actual data, scale: 10 rows
#

l_rows = 10

l_query  = """
   MATCH (n) 
   RETURN n.col1 AS col1, n.col2 AS col2
   LIMIT {0}
   """.format(l_rows)

dd_result1 = my_graph.query(l_query)


print(tabulate(dd_result1, headers='keys', tablefmt='psql'))
print("")
print(type(dd_result1))

#  Sample output,
#
#     +----+--------+-----------------+
#     |    |   col1 | col2            |
#     |----+--------+-----------------|
#     |  0 |      1 | Item number: 1  |
#     |  1 |      6 | Item number: 6  |
#     |  2 |      8 | Item number: 8  |
#     |  3 |     14 | Item number: 14 |
#     |  4 |     17 | Item number: 17 |
#     |  5 |     21 | Item number: 21 |
#     |  6 |     22 | Item number: 22 |
#     |  7 |     26 | Item number: 26 |
#     |  8 |     28 | Item number: 28 |
#     |  9 |     30 | Item number: 30 |
#     +----+--------+-----------------+
#
#     <class 'katana.remote.ResultSet'>


In [ ]:

#  Return (NUM_ROWS: 40,000) rows.
#
#     This as step 1 when we want to validate contents, or use graph data
#     for ML routines, other.
#
#     You should see the Python kernel become unresponsive for a long time.
#


#  l_rows = 40000                                #  My original use case, unresponsive for a long time
#  l_rows = 5000                                 #  Takes 10 (?) seconds, then runs/continues
   #
l_rows = 1000                                    #  This scale works reliably

l_query  = """
   MATCH (n) 
   RETURN n.col1 AS col1, n.col2 AS col2
   LIMIT {0}
   """.format(l_rows)

dd_result2 = my_graph.query(l_query)


l_cntr = 0
   #
for l_each in dd_result2.itertuples():
   l_cntr += 1  
      #
   if (l_cntr < 10):
      print("Counter: %-4d   (%s)   (%s)" % (l_cntr, l_each.col1, l_each.col2))

print("")
print("Number of rows in DataFrame: %d   Value of l_cntr: %d" % (len(dd_result2), l_cntr))
print("--")

#  Sample output,
#
#     Counter: 1      (0)   (Item number: 0)
#     Counter: 2      (5)   (Item number: 5)
#     Counter: 3      (6)   (Item number: 6)
#     Counter: 4      (8)   (Item number: 8)
#     Counter: 5      (9)   (Item number: 9)
#     Counter: 6      (11)   (Item number: 11)
#     Counter: 7      (12)   (Item number: 12)
#     Counter: 8      (13)   (Item number: 13)
#     Counter: 9      (17)   (Item number: 17)
#  
#     Number of rows in DataFrame: 1000   Value of l_cntr: 1000


In [ ]:

#  Use this to see if/when next cell will run
#
print("--")


In [ ]:

%%time 

#  Move the above to paginated fetching, which solves our (hanging, unresponsive) problem.
#

l_rows = 40000
#  l_rows = 5000 
#  l_rows = 10000

l_query  = """
   MATCH (n) 
   RETURN n.col1 AS col1, n.col2 AS col2
   LIMIT {0}
   """.format(l_rows)


dd_result4 = my_graph.query_paginated(l_query)
#
#  Returns,  katana_enterprise.remote.sync_wrappers.PaginatedResultSet
#     It has no length
#     It has no itertuples()
#     It does have a head, maxmimum 40
#
#  The above is true, because of [ paging ]. This is a list of lists.
#  See examples below.


#  l_resultA = dd_result3.head(10)
#  print(type(l_resultA))
      #
#   Maximum value: 40
#
#     print(dd_result3.head(44))
#     >> ValueError: You can only view a maximum of 40 rows.


#  Using a query_paginated()
#
l_cntr1 = 0
l_cntr2 = 0
   #
for l_page in dd_result4:
   l_cntr1 += 1
   for l_each in l_page.itertuples():
      l_cntr2 += 1
         #
      if (l_cntr2 < 10):
        print("Counter: %-4d   (%s)   (%s)" % (l_cntr2, l_each.col1, l_each.col2))

print("")
print("Number of rows in total result set: %d   Number of pages: %d" % (l_cntr2, l_cntr1))
 

#  Again; 2MB of raw data, 40K rows.
#        
#  Sample output,
#
#     Counter: 1      (20178)   (Item number: 20178)
#     Counter: 2      (20182)   (Item number: 20182)
#     Counter: 3      (20183)   (Item number: 20183)
#     Counter: 4      (20185)   (Item number: 20185)
#     Counter: 5      (20187)   (Item number: 20187)
#     Counter: 6      (20188)   (Item number: 20188)
#     Counter: 7      (20189)   (Item number: 20189)
#     Counter: 8      (20200)   (Item number: 20200)
#     Counter: 9      (20201)   (Item number: 20201)
#     
#     Number of rows in total result set: 40000   Number of pages: 3
#     CPU times: user 3min 43s, sys: 79.9 ms, total: 3min 43s
#     Wall time: 3min 46s
        

In [26]:

%%time 

#  Modify the above; add capturing the full result set into a DataFrame
#

#  l_rows = 40000
l_rows = 10

l_query  = """
   MATCH (n) 
   RETURN n.col1 AS col1, n.col2 AS col2
   LIMIT {0}
   """.format(l_rows)


dd_result4 = my_graph.query_paginated(l_query)
   #
pd_data = pd.DataFrame()
   #
l_cntr1 = 0
l_cntr2 = 0
   #
for l_page in dd_result4:
   l_cntr1 += 1
   for l_each in l_page.itertuples():
      l_cntr2 += 1
         #
      pd_data = pd_data.append({l_each}, ignore_index=True)

print("--")


#  print("Number of rows in total result set: %d   Number of pages: %d" % (l_cntr2, l_cntr1))
 

#  Sample output,
#


          0/? [?op/s]

TypeError: cannot concatenate object of type '<class 'set'>'; only Series and DataFrame objs are valid